In [1]:
import pandas as pd

df = pd.read_csv('../tesla_data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2640 entries, 0 to 2639
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Year                  2640 non-null   int64  
 1   Month                 2640 non-null   int64  
 2   Region                2640 non-null   object 
 3   Model                 2640 non-null   object 
 4   Estimated_Deliveries  2640 non-null   int64  
 5   Production_Units      2640 non-null   int64  
 6   Avg_Price_USD         2640 non-null   float64
 7   Battery_Capacity_kWh  2640 non-null   int64  
 8   Range_km              2640 non-null   int64  
 9   CO2_Saved_tons        2640 non-null   float64
 10  Source_Type           2640 non-null   object 
 11  Charging_Stations     2640 non-null   int64  
dtypes: float64(2), int64(7), object(3)
memory usage: 247.6+ KB


In [8]:
bounds = (df
 .describe()
 .transpose()
 .assign(IQR=lambda x: x['75%'] - x['25%'])
 .assign(lower_bound=lambda x: x['25%'] - 1.5 * x['IQR'])
 .assign(upper_bound=lambda x: x['75%'] + 1.5 * x['IQR'])
)

bounds

,count,mean,std,min,25%,50%,75%,max,IQR,lower_bound,upper_bound
Year,2640.0,2020.000000,3.162877,2015.00,2017.000,2020.000,2023.0000,2025.00,6.0000,2008.00000,2032.00000
Month,2640.0,6.500000,3.452707,1.00,3.750,6.500,9.2500,12.00,5.5000,-4.50000,17.50000
Estimated_Deliveries,2640.0,9922.199621,3935.950093,48.00,7292.000,9857.000,12510.2500,25704.00,5218.2500,-535.37500,20337.62500
Production_Units,2640.0,10655.847348,4260.600858,50.00,7828.250,10546.500,13469.0000,28939.00,5640.7500,-632.87500,21930.12500
Avg_Price_USD,2640.0,84907.340330,20123.258036,50003.70,67726.365,85058.510,102373.0425,119965.36,34646.6775,15756.34875,154343.05875
Battery_Capacity_kWh,2640.0,87.059470,20.836265,60.00,75.000,82.000,100.0000,120.00,25.0000,37.50000,137.50000
Range_km,2640.0,500.257576,120.868549,330.00,418.000,470.000,586.2500,719.00,168.2500,165.62500,838.62500
CO2_Saved_tons,2640.0,744.076989,353.221224,3.07,499.620,699.515,943.7650,2548.55,444.1450,-166.59750,1609.98250
Charging_Stations,2640.0,8932.133712,3469.565883,3002.00,5897.750,8901.500,11938.0000,14996.00,6040.2500,-3162.62500,20998.37500


In [16]:
def verificar_outlier(row):
    outliers = []
    for col in bounds.index:
        val = row[col]
        lower = bounds.loc[col, "lower_bound"]
        upper = bounds.loc[col, "upper_bound"]
        
        if val < lower:
            outliers.append(f"{col}")
        elif val > upper:
            outliers.append(f"{col}")

    return ", ".join(outliers) if outliers else "Normal"

df["Outlier_Status"] = df.apply(verificar_outlier, axis=1)
df

,Year,Month,Region,Model,Estimated_Deliveries,Production_Units,Avg_Price_USD,Battery_Capacity_kWh,Range_km,CO2_Saved_tons,Source_Type,Charging_Stations,Outlier_Status
0,2023,5,Europe,Model S,17646,17922,92874.27,120,704,1863.42,Interpolated (Month),12207,CO2_Saved_tons
1,2015,2,Asia,Model X,3797,4164,62205.65,75,438,249.46,Official (Quarter),7640,Normal
2,2019,1,North America,Model X,8411,9189,117887.32,82,480,605.59,Interpolated (Month),14071,Normal
3,2021,2,North America,Model 3,6555,7311,89294.91,120,712,700.07,Official (Quarter),9333,Normal
4,2016,12,Middle East,Model Y,12374,13537,114846.78,120,661,1226.88,Estimated (Region),8722,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2635,2021,10,Middle East,Model Y,13488,14333,113771.10,120,719,1454.68,Official (Quarter),4273,Normal
2636,2019,7,Middle East,Model S,18300,18407,81889.67,82,487,1336.81,Official (Quarter),9833,Normal
2637,2019,9,Asia,Model S,13291,14839,97139.38,100,580,1156.32,Estimated (Region),13709,Normal
2638,2020,5,Asia,Cybertruck,3471,3714,69990.57,60,358,186.39,Interpolated (Month),6754,Normal


In [18]:
df.query('Outlier_Status != "Normal"').head(5)

,Year,Month,Region,Model,Estimated_Deliveries,Production_Units,Avg_Price_USD,Battery_Capacity_kWh,Range_km,CO2_Saved_tons,Source_Type,Charging_Stations,Outlier_Status
0,2023,5,Europe,Model S,17646,17922,92874.27,120,704,1863.42,Interpolated (Month),12207,CO2_Saved_tons
15,2025,3,North America,Cybertruck,17079,18904,111739.13,120,682,1747.18,Official (Quarter),3904,CO2_Saved_tons
86,2020,10,Middle East,Cybertruck,18392,19806,68744.20,120,687,1895.30,Interpolated (Month),8649,CO2_Saved_tons
117,2022,11,Middle East,Model Y,21296,23581,57673.97,60,352,1124.43,Estimated (Region),4630,"Estimated_Deliveries, Production_Units"
123,2020,2,Europe,Cybertruck,19298,21732,119779.45,100,571,1652.87,Interpolated (Month),6843,CO2_Saved_tons
